In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [7]:
column_names = ["Sexo", "Largo", "Diametro", "Altura", "Peso completo", "Peso descascarado", "Peso Vísceras", "Peso cascaron", "Anillos"]
data = pd.read_csv("abalone.data", names=column_names)
print("Numero de datos: %d" % len(data))
data.head()

Numero de datos: 4177


,Sexo,Largo,Diametro,Altura,Peso completo,Peso descascarado,Peso Vísceras,Peso cascaron,Anillos
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [9]:
# Para evitar tener la variable categorica "Sexo", transformamos dicha variable en tres boleanos.
for label in "MFI":
    data[label] = data["Sexo"] == label
del data["Sexo"]

In [10]:
data.head()

,Largo,Diametro,Altura,Peso completo,Peso descascarado,Peso Vísceras,Peso cascaron,Anillos,M,F,I
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,True,False,False
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,True,False,False
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,False,True,False
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,True,False,False
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,False,False,True


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Largo              4177 non-null   float64
 1   Diametro           4177 non-null   float64
 2   Altura             4177 non-null   float64
 3   Peso completo      4177 non-null   float64
 4   Peso descascarado  4177 non-null   float64
 5   Peso Vísceras      4177 non-null   float64
 6   Peso cascaron      4177 non-null   float64
 7   Anillos            4177 non-null   int64  
 8   M                  4177 non-null   bool   
 9   F                  4177 non-null   bool   
 10  I                  4177 non-null   bool   
dtypes: bool(3), float64(7), int64(1)
memory usage: 273.4 KB


In [12]:
data.describe()

,Largo,Diametro,Altura,Peso completo,Peso descascarado,Peso Vísceras,Peso cascaron,Anillos
count,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000
mean,0.523992,0.407881,0.139516,0.828742,0.359367,0.180594,0.238831,9.933684
std,0.120093,0.099240,0.041827,0.490389,0.221963,0.109614,0.139203,3.224169
min,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000
25%,0.450000,0.350000,0.115000,0.441500,0.186000,0.093500,0.130000,8.000000
50%,0.545000,0.425000,0.140000,0.799500,0.336000,0.171000,0.234000,9.000000
75%,0.615000,0.480000,0.165000,1.153000,0.502000,0.253000,0.329000,11.000000
max,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000,29.000000


De esto podemos saber:

1)Ninguna variable tiene minimo igual a cero, expecto altura.
2)No todas se comportan de forma Normal(Mean = Median = Mode) pero se acercan a comportamiento Normal
3)Todas numericas, excepto sexo
4)Distinta escala para cada una

In [14]:
df.hist(figsize=(20,20), grid = True, layout = (2,4), bins = 30)


NameError: name 'df' is not defined